# Generate Heatmaps

In [1]:
import folium
from folium.plugins import MarkerCluster
from folium import IFrame,Popup
import pandas as pd
import geopandas as gpd
from geopandas.tools import sjoin
import shapely
from shapely.geometry import Point
import numpy as np
import pysal as ps

In [2]:
data = pd.read_csv("data/gross_rent_with_population.csv").sort_values(['Zip_Code', 'Samples'], ascending=False)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16580 entries, 0 to 16579
Data columns (total 14 columns):
State_Name    16580 non-null object
Place         16580 non-null object
Zip_Code      16580 non-null int64
ALand         16580 non-null int64
AWater        16580 non-null int64
Lat           16580 non-null float64
Lon           16580 non-null float64
Mean          16580 non-null int64
Median        16580 non-null int64
Stdev         16580 non-null int64
Samples       16580 non-null int64
Population    16580 non-null int64
City          16580 non-null object
County        16580 non-null object
dtypes: float64(2), int64(8), object(4)
memory usage: 1.9+ MB


In [3]:
geo_series = gpd.GeoSeries(data.apply(lambda z: Point(z['Lon'], z['Lat']) , 1), crs={'init': 'epsg:4326'})

geo_data = gpd.GeoDataFrame(data.drop(columns=['Lon', 'Lat']), geometry=geo_series).reset_index(drop=True)
geo_data.info()
geo_data.head(10)

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 16580 entries, 0 to 16579
Data columns (total 13 columns):
State_Name    16580 non-null object
Place         16580 non-null object
Zip_Code      16580 non-null int64
ALand         16580 non-null int64
AWater        16580 non-null int64
Mean          16580 non-null int64
Median        16580 non-null int64
Stdev         16580 non-null int64
Samples       16580 non-null int64
Population    16580 non-null int64
City          16580 non-null object
County        16580 non-null object
geometry      16580 non-null object
dtypes: int64(8), object(5)
memory usage: 1.6+ MB


,State_Name,Place,Zip_Code,ALand,AWater,Mean,Median,Stdev,Samples,Population,City,County,geometry
0,Alaska,Akutan city,99926,343946421,213611590,720,662,283,72,1460,Metlakatla,Prince Wales Ketchikan,POINT (-131.4704235 55.13835200000001)
1,Alaska,Hydaburg city,99922,1506300,5839,483,486,183,21,384,Hydaburg,Prince Wales Ketchikan,POINT (-132.818341 55.20412)
2,Alaska,Craig city,99921,18654310,6010703,1004,980,386,126,1920,Craig,Prince Wales Ketchikan,POINT (-133.111069 55.488233)
3,Alaska,Coffman Cove city,99918,30909225,9766357,802,795,40,15,231,Coffman Cove,Prince Wales Ketchikan,POINT (-132.841933 56.001751)
4,Alaska,Akutan city,99901,12523988591,4646176559,1255,1175,493,220,13508,Ketchikan,Ketchikan Gateway,POINT (-131.1066847 55.4499378)
5,Alaska,Lutak,99840,328819031,16325832,0,0,0,0,968,Skagway,Skagway Hoonah Angoon,POINT (-135.589046 59.412212)
6,Alaska,Akutan city,99835,17109651,32661843,1273,1129,555,117,8880,Sitka,Sitka,POINT (-135.3522511 57.0451454)
7,Alaska,Petersburg,99833,114295601,3884325,931,934,361,382,3202,Petersburg,Wrangell Petersburg,POINT (-132.873808 56.754283)
8,Alaska,Akutan city,99829,16962765542,7798365244,769,725,362,238,777,Hoonah,Skagway Hoonah Angoon,POINT (-135.513003 58.0735321)
9,Alaska,Klukwan,99827,3598502,1300463,669,736,170,5,2602,Haines,Haines,POINT (-135.886877 59.39976600000001)


In [4]:
# Combine data with the same place name
tmp = geo_data.reset_index(drop=True)
tmp = tmp.drop(columns=["Zip_Code", "ALand", "AWater", "Median", "Stdev"])
tmp.head()

,State_Name,Place,Mean,Samples,Population,City,County,geometry
0,Alaska,Akutan city,720,72,1460,Metlakatla,Prince Wales Ketchikan,POINT (-131.4704235 55.13835200000001)
1,Alaska,Hydaburg city,483,21,384,Hydaburg,Prince Wales Ketchikan,POINT (-132.818341 55.20412)
2,Alaska,Craig city,1004,126,1920,Craig,Prince Wales Ketchikan,POINT (-133.111069 55.488233)
3,Alaska,Coffman Cove city,802,15,231,Coffman Cove,Prince Wales Ketchikan,POINT (-132.841933 56.001751)
4,Alaska,Akutan city,1255,220,13508,Ketchikan,Ketchikan Gateway,POINT (-131.1066847 55.4499378)


In [5]:
# Pre-aggregate data on city level
tmp['totals'] = tmp['Mean'] * tmp['Samples']
citywise_data = tmp.dissolve(by=['State_Name','County', 'City'], aggfunc='sum', as_index=False)
citywise_data['Mean'] = (citywise_data.totals / citywise_data.Samples).fillna(0)
citywise_data.geometry = citywise_data.geometry.centroid
citywise_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 12654 entries, 0 to 12653
Data columns (total 8 columns):
State_Name    12654 non-null object
County        12654 non-null object
City          12654 non-null object
geometry      12654 non-null object
Mean          12654 non-null float64
Samples       12654 non-null int64
Population    12654 non-null int64
totals        12654 non-null int64
dtypes: float64(1), int64(3), object(4)
memory usage: 790.9+ KB


In [6]:
# Read counties
counties = gpd.read_file("data/gz_2010_us_050_00_20m/gz_2010_us_050_00_20m.shp").set_index('GEO_ID', drop=False)
join_result = gpd.tools.sjoin(counties, citywise_data.to_crs(counties.crs), op='contains').reset_index()
join_result.head()
join_result.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 12615 entries, 0 to 12614
Data columns (total 16 columns):
index          12615 non-null object
GEO_ID         12615 non-null object
STATE          12615 non-null object
COUNTY         12615 non-null object
NAME           12615 non-null object
LSAD           12615 non-null object
CENSUSAREA     12615 non-null float64
geometry       12615 non-null object
index_right    12615 non-null int64
State_Name     12615 non-null object
County         12615 non-null object
City           12615 non-null object
Mean           12615 non-null float64
Samples        12615 non-null int64
Population     12615 non-null int64
totals         12615 non-null int64
dtypes: float64(2), int64(4), object(10)
memory usage: 1.5+ MB


In [7]:
# Multiply mean and Samples to be able to calculate the mean by county
sum_columns = join_result.groupby('GEO_ID').sum()
counties['Mean'] = (sum_columns['totals'] / sum_columns['Samples']).fillna(0)
counties['Population'] = (sum_columns['Population']).fillna(0)

In [8]:
counties['Population'] = counties['Population'].fillna(0).astype(np.int64)
counties['Mean'] = counties['Mean'].fillna(0).astype(np.int64)
counties.info()
counties.head()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 3221 entries, 0500000US01001 to 0500000US51041
Data columns (total 9 columns):
GEO_ID        3221 non-null object
STATE         3221 non-null object
COUNTY        3221 non-null object
NAME          3221 non-null object
LSAD          3221 non-null object
CENSUSAREA    3221 non-null float64
geometry      3221 non-null object
Mean          3221 non-null int64
Population    3221 non-null int64
dtypes: float64(1), int64(2), object(6)
memory usage: 251.6+ KB


,GEO_ID,STATE,COUNTY,NAME,LSAD,CENSUSAREA,geometry,Mean,Population
index_left,,,,,,,,,
0500000US01001,0500000US01001,01,001,Autauga,County,594.436,"POLYGON ((-86.496774 32.344437, -86.7178969999...",1019,57364
0500000US01009,0500000US01009,01,009,Blount,County,644.776,"POLYGON ((-86.577799 33.765316, -86.7591439999...",624,54010
0500000US01017,0500000US01017,01,017,Chambers,County,596.531,"POLYGON ((-85.18413099999999 32.870525, -85.12...",631,37014
0500000US01021,0500000US01021,01,021,Chilton,County,692.854,"POLYGON ((-86.51734399999999 33.020566, -86.51...",637,33633
0500000US01033,0500000US01033,01,033,Colbert,County,592.619,"POLYGON ((-88.139988 34.581703, -88.139246 34....",681,52812


In [9]:
def add_choropleth(mapobj, gdf, id_field, value_field, legend_name, fill_color = 'YlOrRd', fill_opacity = 0.6, 
                    line_opacity = 0.2, num_classes = 5, classifier = 'Fisher_Jenks'):
    #Allow for 3 Pysal map classifiers to display data
    #Generate list of breakpoints using specified classification scheme. List of breakpoint will be input to choropleth function
    if classifier == 'Fisher_Jenks':
        threshold_scale=ps.esda.mapclassify.Fisher_Jenks(gdf[value_field], k = num_classes).bins.tolist()
    if classifier == 'Equal_Interval':
        threshold_scale=ps.esda.mapclassify.Equal_Interval(gdf[value_field], k = num_classes).bins.tolist()
    if classifier == 'Quantiles':
        threshold_scale=ps.esda.mapclassify.Quantiles(gdf[value_field], k = num_classes).bins.tolist()
    if classifier == 'Percentiles':
        num_classes = 6
        threshold_scale=ps.esda.mapclassify.Percentiles(gdf[value_field], pct=[10, 50, 90, 99, 99.9, 100]).bins.tolist()
    if classifier == 'Jenks_Caspall':
        threshold_scale=ps.esda.mapclassify.Jenks_Caspall(gdf[value_field],  k = num_classes).bins.tolist()
    
    #Convert the GeoDataFrame to WGS84 coordinate reference system
    gdf_wgs84 = gdf.to_crs({'init': 'epsg:4326'})
    
    #Call Folium choropleth function, specifying the geometry as a the WGS84 dataframe converted to GeoJSON, the data as 
    #the GeoDataFrame, the columns as the user-specified id field and and value field.
    #key_on field refers to the id field within the GeoJSON string
    mapobj.choropleth(gdf_wgs84.to_json(), data = gdf,
                columns = [id_field, value_field], key_on = 'feature.properties.{}'.format(id_field),
                fill_color = fill_color, fill_opacity = fill_opacity, line_opacity = line_opacity,  
                threshold_scale = threshold_scale, legend_name = legend_name, highlight=False, smooth_factor=1.0)
    return mapobj

def add_point_clusters(mapobj, gdf, popup_field_list, layer_name):
    #Create empty lists to contain the point coordinates and the point pop-up information
    coords, popups = [], [] 
    #Loop through each record in the GeoDataFrame
    for i, row in gdf.iterrows():
        #Append lat and long coordinates to "coords" list
        coords.append([row.geometry.y, row.geometry.x])
        #Join together the fields in "popup_field_list" with a linebreak between them
        label = '<br>'.join(["{name}: {value}".format(name=field, value=str(row[field]) if type(row[field]) is str or type(row[field]) is int else "{:.1f}".format(row[field])) for field in popup_field_list])
        #Append an IFrame that uses the HTML string to the "popups" list 
        popups.append(Popup(label))
        
    #Create a Folium feature group for this layer, since we will be displaying multiple layers
    pt_lyr = folium.FeatureGroup(name = layer_name)
    
    #Add the clustered points of crime locations and popups to this layer
    pt_lyr.add_child(MarkerCluster(locations = coords, popups = popups))
    
    #Add this point layer to the map object
    mapobj.add_child(pt_lyr)
    return mapobj


In [10]:
m = folium.Map(location=[48, -102], zoom_start=3, control_scale=True, prefer_canvas=True)

m = add_choropleth(m, counties, 'GEO_ID', 'Mean', legend_name='Mean Gross Rent', classifier='Percentiles', num_classes=6)
m = add_point_clusters(m, citywise_data, ['City','Population','Mean'], 'Mean Gross Rent per City')
folium.LayerControl().add_to(m) #Add layer control to toggle on/off
m.save("data/mean_gross_rent_per_county.html")